In [1]:
import import_ipynb
import logging
import numpy as np
from optparse import OptionParser
import os
import sys
from time import time
import matplotlib.pyplot as plt
import random  
from random import sample 
import pandas as pd
import pickle
import scipy
from tempfile import mkdtemp
from shutil import rmtree

from sklearn.compose import make_column_selector
# from sklearn.ensemble import StackingClassifier
from sklearn.compose import ColumnTransformer
from sklearn.preprocessing import LabelBinarizer
from sklearn.model_selection import cross_validate
from sklearn.preprocessing import PolynomialFeatures
from mlxtend.classifier import StackingClassifier, StackingCVClassifier
from sklearn.decomposition import PCA, NMF, KernelPCA, LatentDirichletAllocation
from sklearn.model_selection import GridSearchCV
from sklearn.linear_model import LogisticRegression
from sklearn.base import BaseEstimator, TransformerMixin
from sklearn.feature_selection import SelectKBest, chi2, mutual_info_classif
from sklearn.tree import DecisionTreeClassifier
from sklearn.svm import SVC
from sklearn.ensemble import AdaBoostClassifier, BaggingClassifier
import xgboost as xgb
from catboost import CatBoostClassifier, Pool, cv
from sklearn.model_selection import train_test_split
from sklearn.ensemble import VotingClassifier
from sklearn.datasets import fetch_20newsgroups
from sklearn.feature_extraction.text import TfidfVectorizer
from sklearn.feature_extraction.text import HashingVectorizer
from sklearn.feature_selection import SelectFromModel
from sklearn.feature_selection import SelectKBest, chi2
from sklearn.linear_model import RidgeClassifier
from sklearn.pipeline import Pipeline, FeatureUnion
from sklearn.svm import LinearSVC
from sklearn.linear_model import SGDClassifier
from sklearn.linear_model import Perceptron
from sklearn.linear_model import PassiveAggressiveClassifier
from sklearn.naive_bayes import BernoulliNB, MultinomialNB
from sklearn.neighbors import KNeighborsClassifier
from sklearn.neighbors import NearestCentroid
from sklearn.ensemble import RandomForestClassifier
from sklearn.utils.extmath import density
from sklearn import metrics
from torch import nn
from skorch import NeuralNetClassifier

import My_Transformers
from sklearn.preprocessing import StandardScaler
# Global var
TRAIN_ROW_COUNT = 7964
from My_Transformers import *
from Word2Vec_Transformer import Text_cleaner_transformer
import warnings
warnings.filterwarnings ("ignore")

# Global vars
VERSION = 6.5
trainDF = None
testDF  = None
CACHEDIR = mkdtemp()

importing Jupyter notebook from My_Transformers.ipynb
importing Jupyter notebook from Word2Vec_Transformer.ipynb


2020-09-23 07:29:40,348 INFO adding document #0 to Dictionary(0 unique tokens: [])
2020-09-23 07:29:40,350 INFO built Dictionary(12 unique tokens: ['computer', 'human', 'interface', 'response', 'survey']...) from 9 documents (total 29 corpus positions)


In [2]:
def pickleSave (obj, file):

    if VERSION != '':

        file, ext = os.path.splitext (file)
        file += "_v" + str (VERSION) + ext
    dirs = os.path.dirname (file) 
    if dirs:
        os.makedirs (dirs, exist_ok=True)
    with open (file, 'wb') as f:
        pickle.dump (obj, f)
    return

def unpickle (file):

    if VERSION != '':

        file, ext = os.path.splitext(file)
        file += "_v" + str(VERSION) + ext
    return pickle.load (open (file, 'rb'))

In [3]:
class Discrete2OneHot_FeatureTransformer (BaseEstimator, TransformerMixin):
    
    def __init__(self):
        
        self.LB_1 = None
        self.LB_2 = None
        return

    def fit (self, X, y=None, **fit_params):
        print ('Discrete2OneHot_FeatureTransformer: fit(): X.shape =', X.shape)
        self.LB_1 = LabelBinarizer ()
        self.LB_1.fit (list (X['keyword']))
        self.LB_2 = LabelBinarizer ()
        self.LB_2.fit (list (pd.isnull (X['location'])))
        return self

    def transform (self, X, y=None, **fit_params):
        
        A = self.LB_1.transform (list (X['keyword']))
        B = self.LB_2.transform (list (pd.isnull (X['location']))).reshape (-1, 1)
        X = np.hstack ((A, B))
        print ('Discrete2OneHot_FeatureTransformer: transform(): X.shape =', X.shape)
        return X

# FV Trainers

In [4]:
# Global vars

pca_nmf_ft = ColumnTransformer ([ ('pca_nmf', Pipeline ([
        
        ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
        ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
        ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
        ('standardization',         StandardScaler (with_mean=False)),
        ('pca_nmf',                 PCA_NMF_TrainTest_FeatureTransformer (isSparseOut=True))
], memory=CACHEDIR), 'text')])


tfidf_ft = ColumnTransformer ([ ('tfidf', Pipeline ([

        ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
        ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
        ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
        ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
        ('standardization',         StandardScaler (with_mean=False))
], memory=CACHEDIR), 'text')])


pca_category_ft = ColumnTransformer ([
        ('keyword_category', Discrete2OneHot_FeatureTransformer (), ['keyword', 'location']),
        ('tfidf_pca',        Pipeline ([

            ('tfidf_vectorizer',        TfidfVectorizer (analyzer='char', ngram_range=(3, 7), max_df=0.90, min_df=5, token_pattern='(\S+)')),
            ('sparse_svm_feat_select1', SparseSVM_feature_selector ()),
            ('interactions',            PolynomialFeatures (2, interaction_only=True, include_bias=False)),
            ('sparse_svm_feat_select2', SparseSVM_feature_selector ()),
            ('standardization',         StandardScaler (with_mean=False)),
            ('pca',                     PCA_NMF_TrainTest_FeatureTransformer (isNMF=False, isSparseOut=False))
        ], memory=CACHEDIR), 'text')
])

In [5]:
def init ():
    
    global pca_nmf_ft, tfidf_ft, pca_category_ft, trainDF
    train_file = "Data/all.csv"
    trainDF = pd.read_csv (train_file)
    # df = df[0:20]
    trainDF['text'] = trainDF.text.astype (str)
    trainDF['keyword'] = trainDF.keyword.astype (str)
    trainDF['location'] = trainDF.location.astype (str)
    
    # pca_nmf_ft.fit (trainDF, trainDF['target'])
    # tfidf_ft.fit (trainDF, trainDF['target'])
    # pca_category_ft.fit (trainDF, trainDF['target'])
    return

In [6]:
init ()

# L-0 Classifiers

In [7]:
# Global vars

lsvc_1 = Pipeline ([
    ('pca_nmf_ft', pca_nmf_ft),
    ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
], memory=CACHEDIR)
lsvc_1.name = 'lsvc_1'

lsvc_2 = Pipeline ([
    ('tfidf_ft',   tfidf_ft),
    ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
], memory=CACHEDIR)
lsvc_2.name = 'lsvc_2'

lsvc_3 = Pipeline ([
    ('pca_category_ft', pca_category_ft),
    ('clf',        Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
], memory=CACHEDIR)
lsvc_3.name = 'lsvc_3'

catb_1 = Pipeline ([
    ('pca_nmf_ft', pca_nmf_ft),
    ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
], memory=CACHEDIR)
catb_1.name = 'catb_1'

catb_2 = Pipeline ([
    ('tfidf_ft',   tfidf_ft),
    ('clf',        Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
], memory=CACHEDIR)
catb_2.name = 'catb_2'

catb_3 = Pipeline ([
    ('pca_category_ft', pca_category_ft),
    ('clf',             Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False}))
], memory=CACHEDIR)
catb_3.name = 'catb_3'

fcnn_1 = Pipeline ([
    ('pca_category_ft', pca_category_ft),
    ('clf',             Best_clf_cv_transformer ({ 'name': 'FCNN'}))
], memory=CACHEDIR)
fcnn_1.name = 'fcnn_1'

# Stacking Classifiers

In [8]:
# Global vars
# Level 1

sclf_1_1 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1], 
                               meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
sclf_1_1.name = 'sclf_1_1'

sclf_1_2 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1],
                               meta_classifier=Best_clf_cv_transformer ({ 'name': 'RF',    'params': {'class_weight': 'balanced', 'n_estimators': 3, 'max_features': None} }))
sclf_1_2.name = 'sclf_1_2'

sclf_1_3 = StackingClassifier (classifiers=[lsvc_1, lsvc_2, lsvc_3, catb_1, catb_2, catb_3, fcnn_1],
                               meta_classifier=Best_clf_cv_transformer ({ 'name': 'Catb',  'isCV': False }))
sclf_1_3.name = 'sclf_1_3'

# Level 2

sclf_2_1 = StackingClassifier (classifiers=[sclf_1_1, sclf_1_2, sclf_1_3],
                                 meta_classifier=Best_clf_cv_transformer ({ 'name': 'LSvc',  'params': {'penalty': 'l2', 'class_weight': 'balanced'}, 'param_grid': {'C' : [0.005, 0.01, 0.05, 0.1, 1, 10]} }) )
sclf_2_1.name = 'sclf_2_1'

In [9]:
def train (clf):
    
    clf.fit (trainDF, trainDF['target'])
    return

In [10]:
def predict (clf, iterCount=0):
    
    test_file = "Data/test.csv"
    testDF = pd.read_csv (test_file)
    # testDF = testDF[0:10]
    testDF['text'] = testDF.text.astype (str)
    testDF['keyword'] = testDF.keyword.astype (str)
    testDF['location'] = testDF.location.astype (str)
    
    predictions = clf.predict (testDF)
    testDF['target'] = predictions
    testDF['target'] = testDF.target.astype (int)
    submitDF = testDF[['id', 'target']]
    submitDF.to_csv ('submission_v'+str (VERSION) + clf.name + '.' + str (iterCount+1) + '.csv', index=False)
    return

In [11]:
# init ()
# train (sclf_2_1)
# predict (sclf_2_1)

In [12]:
def main (clf, threshold=0.9, maxIterCount=10, thresh_depreciation=0.01):

    global trainDF, sclf_2_1
    # init ()
    newTrainAddCount = 1
    iterCount = 0
    while newTrainAddCount > 0:

        threshold = threshold - iterCount * thresh_depreciation
        if iterCount >= maxIterCount:
            break
        train (clf)
        predict (clf, iterCount)
        Pr = sclf_2_1.predict_proba (trainDF)
        newTrainAddCount = 0
        for idx, row in trainDF.iterrows ():
            if not pd.isnull (row['target']):

                continue
            print ('considering idx =', idx)
            pr = Pr[idx]
            p  = 0
            if pr[0] > 0.5:
                pr = pr[0]
                p  = 0
            else:
                pr = pr[1]
                p  = 1
            if pr >= threshold:
                # add in the training set
                trainDF.at[idx, 'target'] = p
                newTrainAddCount += 1
        # arrange df such that 1st rows with y=0/1 then rows with y=NaN
        arrangedDF1 = trainDF[pd.isnull (trainDF['target']) == False]
        arrangedDF2 = trainDF[pd.isnull (trainDF['target'])]
        trainDF = pd.concat ([arrangedDF1, arrangedDF2], axis=0)
        iterCount += 1
    trainDF.to_csv ('Data/all_'+str (VERSION) + clf.name + '.' + str (iterCount+1)+'.csv', index=False)
    train (clf)
    predict (clf, iterCount)

    # CV for the bclfs on the original train rows
    print ('------------------- CV for the sclf_2_1 on the original train rows -------------------')
    trainDF = trainDF[:TRAIN_ROW_COUNT]
    cv_results = []
    cv_result = cross_validate (sclf_2_1, trainDF, cv=5)
    cv_score = np.mean (cv_result['test_score'])
    print ()
    print (clf.name, cv_score)
    print (cv_result)
    rmtree (CACHEDIR)
    return

In [1]:
main (sclf_2_1, maxIterCount=1)

# clean file
import re
text_cleaner_transformer = Text_cleaner_transformer ()
files = ["Data/all.csv", "Data/train.csv", "Data/test.csv"]
for file in files:
    df = pd.read_csv (file)
    df['text'] = text_cleaner_transformer.transform (df['text'])
    df.to_csv (file, index=False)

# Findings

LinearSVC with L1-based feature selection for X.shape = (7964, 83758)

--------------------- Best parameter (CV score=0.693):

{'feature_selection__estimator__tol': 0.001}

New #features =  1393

LinearSVC with L1-based feature selection for X.shape = (7964, 970921)

--------------------- Best parameter (CV score=0.765):

{'feature_selection__estimator__tol': 0.0001}

New #features =  1374

Find optimal PCA dims and the same no. of NMF features for X.shape = (11227, 1374)

--------------------- Best:  parameters = (0.9, 0.1) , CV = 0.8258456437870457

PCA dimensionality, explainedVarRatio =  784 0.9

training LSvc for X.shape = (7964, 1568)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.829):

{'C': 0.005}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1374)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.818):

{'C': 0.005}
Done Fitting LSvc
Discrete2OneHot_FeatureTransformer: fit(): X.shape = (11227, 2)
Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Find optimal PCA dims for X.shape = (11227, 1374)

--------------------- Best:  parameters = (0.9, 0.1) , CV = 0.8258456437870457

PCA dimensionality, explainedVarRatio =  784 0.9
training LSvc for X.shape = (7964, 1007)
LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.826):

{'C': 0.005}

Done Fitting LSvc

training Catb for X.shape = (7964, 1568)
Done Fitting Catb

training Catb for X.shape = (7964, 1374)

Done Fitting Catb

training Catb for X.shape = (7964, 1007)

Done Fitting Catb

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)


Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

training LSvc for X.shape = (7964, 6)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.973):

{'C': 1}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1568)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.829):

{'C': 0.005}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1374)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.818):

{'C': 0.005}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1007)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.826):

{'C': 0.005}

Done Fitting LSvc

training Catb for X.shape = (7964, 1568)

Done Fitting Catb

training Catb for X.shape = (7964, 1374)

Done Fitting Catb

training Catb for X.shape = (7964, 1007)

Done Fitting Catb

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

training RF for X.shape = (7964, 6)

RF : Best_clf_cv_transformer: starting CV = 5

RF : cv_score:   0.973

Done Fitting RF

training LSvc for X.shape = (7964, 1568)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.829):

{'C': 0.005}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1374)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.818):

{'C': 0.005}

Done Fitting LSvc

training LSvc for X.shape = (7964, 1007)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.826):

{'C': 0.005}


Done Fitting LSvc

training Catb for X.shape = (7964, 1568)

Done Fitting Catb

training Catb for X.shape = (7964, 1374)

Done Fitting Catb

training Catb for X.shape = (7964, 1007)

Done Fitting Catb

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

training Catb for X.shape = (7964, 6)

Done Fitting Catb


Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

Discrete2OneHot_FeatureTransformer: transform(): X.shape = (11227, 223)

training LSvc for X.shape = (7964, 3)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.974):

{'C': 0.005}

Done Fitting LSvc

In the next iteration the final output CV score improved as:

training LSvc for X.shape = (11471, 3) (11471,)

LSvc : Best_clf_cv_transformer: Best parameter (CV score=0.979):

# without any cleaning, this scored 80 % in the kaggle test !